# Epipolar Geometry

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



## Opening Assumptions

To solve this problem one step at a time we need to make some assumptions.

- We have to assume that we are viewing a static scene and that we have two different views of this scene.
- We will assume that we already have a set of point correspondences in our two views. \\How we did this is not our concern in this section.
- We assume that we know the intrinsic parameters of the camera. And we will assume that they are the same for both views.


## What we don't know
Even with these assumptions we still have a problem on our hands. 

We know a set of 2D points but we don't know the extrinsic parameters and we don't know the 3D coordinates that correspond to the points in the images.

This leads to a bit of a catch-22.
To find the extrinsic parameters (rotation and translation) we need the 3D points.
To find the 3D points we need to know the extrinsic camera parameters.

## Route to solution

- Disentangle the 3D coordinates from the camera motion (rotation/translation) algebraically.
- Remove the 3D coordinates from the algebra so that we have equations in 2D image coordinates only.
- Use these to solve for camera motion (using the 8-point algorithm).
- Now we have the extrinsic parameters we can determine the 3D coordinates. 
- The 3D coordinates are the reconstruction.
	

**Note: that this is solving a neat mathematics problem. So a gentle reminder: the real world is not a neat mathematics problem.**

## Epipolar Geometry
We will continue to call a point in the 3D world $X$.

The projection of this into the first view results in 2D coordinate $x_1$ and its projection into the second view results in 2D coordinate $x_2$.

We refer to the center of projection of the two cameras as $O_1$ and $O_2$.

If we draw a line beteen the two centers of projection this line will intersect the two image planes, the points of intersection are called the epipoles $e_1$ for the intersection with the first view plane and $e_2$ for the intersection with image plane of the second view.

Note: these intersections do not have to occur inside the small rectangle of the image. The image planes are assumed to continue forever in all directions.
 

![](images/epipolarMuSh.png)
[Image Credit: Mubarak Shah UCF](https://youtu.be/1X93H_0_W5k?t=1890)
 

A triangle is formed between the 3D point $X$ and the two camera centers. This triangle is said to lie on the epipolar plane.
The intersection of the epipolar plane with each image plane is called the epipolar lines $l_1$ and $l_2$.
There is a single epipolar plane for each 3D point $X$

In [4]:
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import ipywidgets as widgets
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec

camera_coords = np.array(np.zeros([2,2]))
epipole_coords = np.array(np.zeros([2,2]))
lambda1 = 0
lambda2 = 0
x1 = 0
x2 = 0



def update_2d_plots(fig,gs, camera_coords, epipole_coords, ep):
    
    #fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    #axs = [fig.add_subplot(1, 3, 2), fig.add_subplot(1, 3, 3)]
    axs = [fig.add_subplot(gs[0,3]),fig.add_subplot(gs[1,3])]
    # Clear existing plots
    axs[0].cla()
    axs[1].cla()

    # Update and configure the first plot (Camera 1 View)
    axs[0].set_title('Camera 1 View')
    axs[0].set_xlim(-1, 1)
    axs[0].set_ylim(-0.5, 0.5)
    rect1 = patches.Rectangle((-0.6, -0.4), 1.2, 0.8, color=(0, 1, 1, 0.2))  # Cyan color
    axs[0].add_patch(rect1)
    axs[0].scatter(camera_coords[0][0], camera_coords[0][1], color='magenta')
    axs[0].text(camera_coords[0][0], camera_coords[0][1], "x1", color='black')
    if ep:
        axs[0].scatter(epipole_coords[0][0], epipole_coords[0][1], color='black', marker='x')
        axs[0].plot([camera_coords[0][0], epipole_coords[0][0]], [camera_coords[0][1], epipole_coords[0][1]], color='magenta')
    
    

    # Update and configure the second plot (Camera 2 View)
    axs[1].set_title('Camera 2 View')
    axs[1].set_xlim(-1, 1)
    axs[1].set_ylim(-0.5, 0.5)
    rect2 = patches.Rectangle((-0.6, -0.4), 1.2, 0.8, color=(1, 1, 0, 0.2))  # Yellow color
    axs[1].add_patch(rect2)
    axs[1].scatter(camera_coords[1][0], camera_coords[1][1], color='green')
    axs[1].text(camera_coords[1][0], camera_coords[1][1], "x2", color='black')
    if ep:
        axs[1].scatter(epipole_coords[1][0], epipole_coords[1][1], color='black', marker='x')
        axs[1].plot([camera_coords[1][0], epipole_coords[1][0]], [camera_coords[1][1], epipole_coords[1][1]], color='green')
    # Redraw the plots
    plt.draw()


# Define a function to update the plot with both elevation and azimuth angles
def update_plot(elev_angle, azim_angle, roll_angle,Xw, Yw, Zw, FL2, Alpha, Beta, Gamma, tx, ty, tz, ep):
    # Create a new matplotlib figure and axis
    fig = plt.figure(figsize=(20, 6))
    gs = GridSpec(2, 4, figure=fig)
    #fig = plt.figure(figsize=(30, 10))
    #ax = fig.add_subplot(111, projection='3d')
    ax = fig.add_subplot(gs[0:4,:], projection='3d')
    # Set axes labels and limits
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_xlim([-2, 2])
    ax.set_ylim([-2, 2])
    ax.set_zlim([0, 4])
    global E_mat
    global rot


    # Second camera array
    K=np.array([[1,0,0,0],
               [0,1,0,0],
               [0,0,1,0],
               [0,0,0,1]])
    
    T = np.array([[1,0,0,tx],
                   [0,1,0,ty],
                   [0,0,1,tz],
                   [0,0,0,1]])
    
    # Individual Euler angle matrices
    alphaRot = np.array([[1,0,0,0],
       [0,math.cos(math.pi*Alpha/180),-math.sin(math.pi*Alpha/180),0],
       [0,math.sin(math.pi*Alpha/180),math.cos(math.pi*Alpha/180),0],
       [0,0,0,1]])
    betaRot = np.array([[math.cos(math.pi*Beta/180),0,math.sin(math.pi*Beta/180),0],
       [0,1,0,0],
       [-math.sin(math.pi*Beta/180),0,math.cos(math.pi*Beta/180),0],
       [0,0,0,1]])
    gammaRot = np.array([
       [math.cos(math.pi*Gamma/180),-math.sin(math.pi*Gamma/180),0,0],
       [math.sin(math.pi*Gamma/180),math.cos(math.pi*Gamma/180),0,0],
        [0,0,1,0],
       [0,0,0,1]])
    # Full rotation matrix but keep in mind that changing the order will change the rotation.
    rot = alphaRot @ betaRot @ gammaRot
    
    # Camera two focal length only.
    K_FL = ([[FL2,0,0,0],
       [0,FL2,0,0],
       [0,0,FL2,0],
       [0,0,0,1]])
    
    '''Special matrix for the applying the focal length to the z-axis only 
    This is used to move the image sensor with the focal length but not resize the sensor
    '''
    K_plane = ([[1,0,0,0],
               [0,1,0,0],
               [0,0 ,FL2,0],
               [0,0 ,0,1]])
    
    '''K_NF is the camera two matrix but without the focal length
    This is to all the red, green and blue axes for camera two 
    to be the same size as for camera one. So this matrix is to help 
    with the visualisation only'''
    K_NF = K @ T @ rot 
    
    '''K_z is for the visualisation only. It allows the camera two frame to be shown in the correct
    position without re-sizing the frame. Note, as we are only affecting the z-axis, ordering matters here.
    You must do the rotation and translation first and only then extend the z-axis or otherwise you will rotate
    and translate what you did to the z-axis and point it in another direction'''
   
    K_z = K @ T  @ rot @ K_plane 
   
    '''This is the full camera two matrix (relative to camera one). The focal length is in multiples 
    of the first camera focal length. Hence the first camera focal lenght is fixed at 1 and therefore all 
    coordinates are in units of the focal length of camera one'''
    
    K = K  @ T  @  rot @  K_FL  
    
       
    # Plotting the axes for the two cameras
    axes = np.array([[[-.1, 0, 0],[.1, 0, 0]],
            [[0, -.1, 0], [0, .1, 0]],
            [[0, 0, 0], [0, 0, 0.5]]])
               
       
    axes_cam_2 = axes.reshape(6,3)
    axes_cam_2 = np.hstack([axes_cam_2, np.ones((6, 1))])
    axes_cam_2 = K_NF @ axes_cam_2.transpose()
    axes_cam_2 = axes_cam_2.transpose() 
    # Remove the last column
    axes_cam_2 = axes_cam_2[:, :-1]
    axes_cam_2 = axes_cam_2.reshape(3,2,3)
    colors = ['r', 'g', 'b']  # Colors for each axis
    for i in range(0, 3):
        ax.plot([axes_cam_2[i][0][0], axes_cam_2[i][1][0]],  # X coordinates
            [axes_cam_2[i][0][1], axes_cam_2[i][1][1]],  # Y coordinates
            [axes_cam_2[i][0][2], axes_cam_2[i][1][2]],  # Z coordinates
            color=colors[i]) 
        
        ax.plot([axes[i][0][0], axes[i][1][0]],  # X coordinates
            [axes[i][0][1], axes[i][1][1]],  # Y coordinates
            [axes[i][0][2], axes[i][1][2]],  # Z coordinates
            color=colors[i])   
     
    
    
    # adding the world coordinate point
    world_coord = np.array([Xw, Yw, Zw])
    ax.scatter(*world_coord, color='black')
    ax.text(Xw, Yw, Zw, "X", color='black')

    # Drawing a line from the origin to the  World coordinate point
    ax.plot([0, world_coord[0]], [0, world_coord[1]], [0, world_coord[2]], color='magenta')
    
    # Creating a plane normal to the y-axis centered at (0, 1, 0)
    x = np.linspace(-.6, .6, 10)
    y = np.linspace(-.4, .4, 10)
    X, Y = np.meshgrid(x, y)
    Z = np.ones_like(X)  # Plane centered at Z=focal length
    image_plane1 = np.array([X,Y,Z, np.ones_like(X)])
    
    
    
    camera_2_center = K_NF @ np.array([0,0,0,1])
    # Drawing a line from the camera 2 center to the point
    ax.plot([camera_2_center[0], world_coord[0]], [camera_2_center[1], world_coord[1]], [camera_2_center[2], world_coord[2]], color='green')
    
    # Adding the plane with transparency
    ax.plot_surface(image_plane1[0], image_plane1[1], image_plane1[2], color='cyan', alpha=0.2)
    
    # This reshapes image_plane1 for matrix multiplication by our camera 2 matrix
    image_plane2 = K_z @ image_plane1.reshape(4,-1) 
    
    # Reshaping back to original shape
    image_plane2 = image_plane2.reshape(image_plane1.shape) 
    ax.plot_surface(image_plane2[0], image_plane2[1], image_plane2[2], color='yellow', alpha=0.2)
    
    # The intersection point where the magenta line intersects the image_plane1 Z = 1
    intersection_point = (Xw/Zw, Yw/Zw, Zw/Zw)
    cam_1_coord = np.array(intersection_point[:2])
    
    
    ax.scatter(*intersection_point, color='magenta')
    world_hom = np.array([Xw,Yw,Zw,1])
    try:
        K_inv = np.linalg.inv(K)
    except np.linalg.LinAlgError:
        print("The matrix is not invertible.")
        
    temp_world = K_inv @ world_hom
    
    intersection_point_imageP2 = np.array([FL2*temp_world[0]/temp_world[2], 
                                  FL2*temp_world[1]/temp_world[2], 
                                  FL2*temp_world[2]/temp_world[2],1])
    
    x2 = intersection_point_imageP2[:3]
    cam_2_coord = intersection_point_imageP2[:2]
    
    intersection_point_imageP2 = K_NF @ intersection_point_imageP2
    pt = (intersection_point_imageP2[0],intersection_point_imageP2[1],intersection_point_imageP2[2])
    ax.scatter(*pt, color='green')
    
    # draw line between camera centers
    ax.plot([0, camera_2_center[0]], [0, camera_2_center[1]], [0, camera_2_center[2]], color='cyan')
    
    points = np.array([[0, 0, 0],  # Origin - camera 1 center
                       [world_coord[0], world_coord[1], world_coord[2]],  # World coordinate
                       [camera_2_center[0], camera_2_center[1], camera_2_center[2]]])  # Camera 2 center

    # Shade in the Epipolar plane
    epipoloar_plane = Poly3DCollection([points])
    epipoloar_plane.set_color('grey')
    epipoloar_plane.set_alpha(0.2)  # Adjust transparency here
    ax.add_collection3d(epipoloar_plane)
    
    # Show the epipole for camera 1
    cam_1_epipole = (camera_2_center[0]/camera_2_center[2], 
                     camera_2_center[1]/camera_2_center[2], 
                     camera_2_center[2]/camera_2_center[2])
    ax.scatter(*cam_1_epipole, color='black', marker='x')
    epipole_coords[0] = np.array([cam_1_epipole[0], cam_1_epipole[1]])
    
    # Show the epipole for camera 2
    cam_2_view_origin = K_inv @ np.array([0,0,0,1])
    cam_2_epipole = (cam_2_view_origin[0]/cam_2_view_origin[2], 
                     cam_2_view_origin[1]/cam_2_view_origin[2],
                     cam_2_view_origin[2]/cam_2_view_origin[2], 1)
    epipole_coords[1] = np.array([cam_2_epipole[0], cam_2_epipole[1]])
    cam_2_epipole = K @ cam_2_epipole
    ax.scatter(*cam_2_epipole[:3], color='black', marker='x')
    
    # Adjust view
    ax.view_init(elev=elev_angle, azim=azim_angle, roll=roll_angle)
    
    #show view in camera 1    
    cam_1_coord = np.array([world_coord[0]/world_coord[2], world_coord[1]/world_coord[2]])
    
    
    camera_coords[0] = cam_1_coord
    camera_coords[1] = cam_2_coord
    
    
    
    update_2d_plots(fig,gs, camera_coords, epipole_coords, ep)
    
    lambda1 = world_coord[2]#math.sqrt((world_coord[0]**2)+(world_coord[1]**2)+(world_coord[2]**2))
    lambda2 = FL2*temp_world[2]# math.sqrt((world_coord[0]-camera_2_center[0])**2+(world_coord[1]-camera_2_center[1])**2+(world_coord[2]-camera_2_center[2])**2)
    print(f'lambda1:{lambda1}')
    print(f'lambda2:{lambda2}')
    
    x1 = np.append(cam_1_coord, 1)
    print(f'x1:{x1}')
    #np.append(cam_2_coord, 1)
    print(f'x2:{x2}')
    Tx = np.array([[0, -tz, ty],
                   [tz, 0, -tx],
                   [-ty, tx, 0]])
    
    R = np.array(rot[:3,:3])
    print(f'x1TxRx2:{ x1 @ Tx @ R @ x2}')
    print(f'lambda1*x1:{lambda1*x1}')
    R_inv = np.linalg.inv(R)
    print(f'R(lambda1*x1)+T:{R_inv @ (lambda1*x1 -np.array([tx, ty,tz])) }' )
    print(f'lambda2*x2:{lambda2*x2}')
    
    E_mat= Tx @ R
    
    
    print(f'Determinant of E=TxR: {np.linalg.det(E_mat)}')
    # Show the plot
    plt.show()
    



elev_slider = widgets.IntSlider(min=-180, max=180, step=1, value=0, description='Elevation')
azim_slider = widgets.IntSlider(min=-180, max=180, step=1, value=90, description='Azimuth')
roll_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-90, description='Roll')

# Sliders for world coordinates
Xw_slider = widgets.FloatSlider(min=-2, max=2, step=0.1, value=0, description='Xw')
Yw_slider = widgets.FloatSlider(min=-2, max=2, step=0.1, value=0.5, description='Yw')
Zw_slider = widgets.FloatSlider(min=0, max=5, step=0.1, value=3, description='Zw')

FL_slider2 = widgets.FloatSlider(min=0.1, max=3, step=0.1, value=1.0, description='Cam 2 Focal')

alpha_slider = widgets.IntSlider(min=-180, max=180, step=1, value=0, description='Cam2 Alpha')
beta_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-150, description='Cam2 Beta')
gamma_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-70, description='Cam2 Gamma')


tx_slider = widgets.FloatSlider(min=-2.0, max=2.0, step=0.1, value=1, description='Tx')
ty_slider = widgets.FloatSlider(min=-2.0, max=2.0, step=0.1, value=1, description='Ty')
tz_slider = widgets.FloatSlider(min=0.0, max=5.0, step=0.1, value=4, description='Tz')

# Group sliders into two columns
left_box = widgets.VBox([elev_slider, azim_slider, roll_slider, Xw_slider, Yw_slider, Zw_slider ])
right_box = widgets.VBox([ FL_slider2, alpha_slider, beta_slider, gamma_slider, tx_slider, ty_slider, tz_slider])

epipoles_checkbox = widgets.Checkbox(value=False, description='Show Epipoles in 2D',disabled=False)

# Combine the two columns into a single horizontal layout
ui = widgets.HBox([left_box,  right_box])


# Interactive widget
out = widgets.interactive_output(update_plot, {'elev_angle': elev_slider, 'azim_angle': azim_slider, 
                                               'roll_angle': roll_slider, 'Xw': Xw_slider, 
                                               'Yw': Yw_slider, 'Zw': Zw_slider, 'FL2': FL_slider2, 
                                               'Alpha': alpha_slider, 'Beta': beta_slider, 'Gamma': gamma_slider,
                                              'tx': tx_slider, 'ty': ty_slider, 'tz': tz_slider, 'ep': epipoles_checkbox})

sliders_box = widgets.VBox([elev_slider, azim_slider, roll_slider, 
                            Xw_slider, Yw_slider, Zw_slider,
                            FL_slider2, alpha_slider, beta_slider, 
                            gamma_slider, tx_slider, ty_slider, tz_slider, epipoles_checkbox])
ui = widgets.HBox([sliders_box, out])


# Display the UI and the output widget
display(ui)

## Null Spaces

A quick linear algebra reminder. A 2D plane in a 3D space is called a subspace (as long as it passes through the origin).

We can define the plane with vectors that lie in the plane. Two independent vectors in the plane will be enough.

However we can also define the plane by a single vector that is orthogonal to it, i.e. a vector that is in its null space.

Keep this in mind as we proceed.

**Ideas like null space, singular matrices, rank deficiency, zero determinant, SVD (Singular Value Decomposition) are all relevant here.**

## Simplifying the Intrinsic Parameters
Firstly let's deal with the intrinsic parameters. 

We will assume no skew in the pixels and assume they are 1:1 aspect ratio. We will assume the focal length is 1. 

This idea of setting the focal length to one is just a way of saying that all other measures will be in units of the focal length. 

So rather than giving sizes in mm, cm or meters we instead give everything in units of the focal length. We can then convert everything easily if we know the focal length. 

We will assume that the origin in the center of the frame.

So the Camera Intrinsic parameter matrix will be 
$$K= \begin{bmatrix}
                fs_x    & 0 & O_x  \\
                0     & fs_y & O_y  \\
                0     & 0 & 1  
            \end{bmatrix} = \begin{bmatrix}
                1    & 0 & 0  \\
                0     & 1 & 0  \\
                0     & 0 & 1  
            \end{bmatrix}$$

Which is the identity matrix, and this allows us to leave it out entirely in our mathematical manipulation.

   

## Epipolar Constraints
So $x_1$ is the projection of the world 3D coordinate point onto the first camera image plane.
$x_1$ will be in homogeneous coordinates.

We will work with everything being relative to the first camera view. Therefore there will be zero rotation and translation for the mathematical description of the first camera view.

\begin{equation}
	\lambda_1x_1 = X
\end{equation}


For the second camera view, we must describe this in terms of camera one. 

This means a Rotation and translation of the 3D point followed by the projection.
\begin{equation}
	\lambda_2x_2=RX+T
\end{equation}
Subsititute equation for the first view into the equation for the second view.

\begin{equation}
	\lambda_2x_2=R(\lambda_1x_1)+T
\end{equation}

## $T_{\times}$

Now it's convenient to remove the $+T$ out of this equation. So what we will do is multiply across by $T_{\times}$.

Remember how we defined this. Take a  vector $T$ and make it into a skew symmetric matrix that performs the cross product with $T$
\begin{equation}
	\lambda_2T_{\times}x_2=T_{\times}R(\lambda_1x_1)+T_{\times}T
\end{equation}

\begin{equation}
	T_{\times}T = 0
\end{equation}

So we can rewite equation for the second view as follows
\begin{equation}
	\lambda_2T_{\times}x_2=\lambda_1T_{\times}Rx_1
\end{equation}
Remember that  $\lambda_1$ and $\lambda_2$ are simply scalar values and can be moved about more freely than vectors or matrices.\\
   

## Getting rid of the $\lambda$s
Now this next step takes a bit of explaining.

Firstly be aware that $T_{\times}x_2$ will result in vector that is orthogonal to $x_2$. So if we get the dot product of $x_2$ with this vector we will get zero.
So multiply across by $x_2$
\begin{equation}
	\lambda_2x_2^{\top}T_{\times}x_2=\lambda_1x_2^{\top}T_{\times}Rx_1
\end{equation}

Which is 
\begin{equation}
	0=\lambda_1x_2^{\top}T_{\times}Rx_1	
\end{equation}

That's $\lambda_2$ gone. Now divide both sides by $\lambda_1$
\begin{equation}
    0=	x_2^{\top}T_{\times}Rx_1	
\end{equation}

## The Epipolar Constraint

\begin{equation}
    x_2^{\top}T_{\times}Rx_1=0
\end{equation}

Is called the Epipolar Constraint.

It is an important result as it relates 2D coordinates without mention of 3D coordinates.

Remember our issue from earlier, the catch-22.
In order to determine camera motion we needed the 3D coordinates and in order to determine the 3D coordinates we needed the camera motion.

The epipolar constraint allows us to determine camera motion without 3D coordinates.

So from there we can work towards getting the 3D coordinates.
    

## The Essential matrix

If we take the central part of the epipolar constraint, the part that doesn't include the two 2D coordinates we have what is called the essential matrix.
\begin{equation}
	E = T_{\times}R \quad \in \mathbb{R}^{3\times3}
\end{equation}

Due to this name the epipolar constraint may be variously called the the essential constraint or the bilinear constraint.

  
## The Epipolar Plane
$E$ is a $3\times3$ matrix of rank 2 which means it has a left and right null space of 1. 
The epipolar constraint stipulates that the  triangle ($\vec{O_1X},\vec{O_2O_1},\vec{O_2X})$ lies on a plane. 

Or we can say that those points alone can define the plane.

Now, as these three are vectors, we can define a triple product with them.

Remember that the triple product defines a volume. And a plane should have a volume of zero.
So we can say
\begin{equation}
	x^{\top}_2(T_{\times} Rx_1) = 0 
\end{equation}
   

The plane is shown in grey in the interactive demonstration.

**However, this all depends on perfect mathematics. Once you move it into a computer, 
the small amounts of round-off error cause this relationship to be a very small number rather than zero**
**Take a look at the values for this and the determinant of $T_{\times} R$ in the interactive demo. They are rarely perfectly zero**
**More on this later**

## A comment on invertability

So we mentioned that $E$ has rank 2.

Therefore it is singular and not invertible.

Imagine you are given the epipolar constraint and asked to work back to get the $\lambda$s.

Can you do so?

No, because you now have zero on the other side of the equation.

This tells you that you have lost some things along the way.
You may remember that rotation and translation (in 3D) has 6-DoF. 

The fact that we have lost information along the way means we cannot get away with just six equations to solve for these.

We will need eight sets of corresponding points and hence this is called the 8-point algorithm.

## Some properties of $E$

This is all great, but the truth is that the epipolar constraint is not really a nice identity.
$E$ is a $3\times3$ matrix. 
If we find enough point correspondences we should be able to recover $E$, but then what?

We don't really want $E$, we want $R$ and $T$ so that we can tell how the camera moved between frames. 
How do we separate out $R$ and $T$?

The space of all essential matrices is called the essential space defined as follows

\begin{equation}
	\mathcal{E} \equiv \left\{ T_{\times}R | R \in SO(3), T \in \mathbb{R}^3 \right\} \subset \mathbb{R}^{3\times3}
\end{equation}

    

## Some properties of $E$

A nonzero matrix $E \in \mathbb{R}^{3\times3}$ is an essential matrix (if and only if) $iff \quad E$ has a singular value decomposition (SVD) $E=U\Sigma V^{\top}$ with 

\begin{equation}
	\Sigma = \begin{bmatrix}
\sigma & 0 & 0\\
0 &\sigma & 0\\
0 & 0 & 0
\end{bmatrix}
\end{equation}

for some $\sigma>0$ and $U,V \in SO(3)$.

This is from the 1989 theorem _Characterization of a the essential Matrix_ by Huang & Faugeras.

And it poses quite a problem because the space of essential matrices is not a linear one, so solving it by the normal linear algebra will find us a $3\times3$ matrix, but it is unlikely to have these required properties.

To add insult to injury here, even if we find the essential matrix, there are two possible decompositions of $R$ and $T$.

The one bit of good news is that in general only one of the decompositions makes sense, i.e. gives positive depth values. Negative depth values would be behind the camera.

## Two decompositions

From the theorem _Pose recovery from the Essential Matrix_ - page 84 An invitation to 3D Vision by Ma, Kosecka, Soatto, Sastry.

There are two relative poses $(R,T)$ with $R\in SO(3)$ and $T\in\mathbb{R}^3$ corresponding to an essential matrix $E\in\mathcal{E}$


For $E=U\Sigma V^{\top}$ we have:
\begin{equation}
	(T_{1\times},R_1) = (UR_{Z(+\frac{\pi}{2})}\Sigma U^{\top}, UR^{\top}_{Z(+\frac{\pi}{2})}V^{\top})
\end{equation}
\begin{equation}
	(T_{2\times},R_2) = (UR_{Z(-\frac{\pi}{2})}\Sigma U^{\top}, UR^{\top}_{Z(-\frac{\pi}{2})}V^{\top})
\end{equation}

In [663]:
#In this code I am using the E matrix that I created from a known rotation and translation matrix
# For the eight point algorithm we must determine the E matrix
U, S, Vh = np.linalg.svd(E_mat)
theta = np.pi / 2  # 90 degrees in radians
S[2] = 0

Rz = np.array([[np.cos(theta), -np.sin(theta), 0],
                [np.sin(theta), np.cos(theta), 0],
                [0, 0, 1]])
Rzneg = np.array([[np.cos(-theta), -np.sin(-theta), 0],
                [np.sin(-theta), np.cos(-theta), 0],
                [0, 0, 1]])
Tx1 = U @ Rz @ np.diag(S) @ U.T
Tx2 = U @ Rzneg @ np.diag(S) @ U.T
T1 = np.array([Tx1[2, 1], Tx1[0, 2], Tx1[1, 0]])
T2 = np.array([Tx2[2, 1], Tx2[0, 2], Tx2[1, 0]])
print(f'T1: {T1}')
print(f'T2: {T2}')

R1 = U @ Rz.T @ Vh
R2 = U @ Rzneg.T @ Vh
print(f'R1:{R1}')
print(f'R2:{R2}')
print(f'Actual Rotation: {rot}')

T1: [-1. -1. -4.]
T2: [1. 1. 4.]
R1:[[ 0.235  0.97   0.06 ]
 [ 0.878 -0.186 -0.44 ]
 [-0.416  0.156 -0.896]]
R2:[[-0.296 -0.814 -0.5  ]
 [-0.94   0.342  0.   ]
 [ 0.171  0.47  -0.866]]
Actual Rotation: [[-0.296 -0.814 -0.5    0.   ]
 [-0.94   0.342  0.     0.   ]
 [ 0.171  0.47  -0.866  0.   ]
 [ 0.     0.     0.     1.   ]]


## Getting an essential matrix
As mentioned earlier our standard linear Algebra methods will recover a $3\times3$ matrix but this is unlikely to meet the stringent criteria of an Essential matrix.

We have two options:

- Recover whatever $3\times3$ matrix we can from our linear methods and then project that on to the space of essential matrices. In other words use the closest essential matrix to the one we recover (whatever closest means) - Easy but lacking accuracy. 
- Optimise the epipolar constraints in the essential space $\mathcal{E}$, accurate but requires non-linear constrained optimisation which is difficult and would require a whole new toolset of skills.


We will use the first approach.
    

In [6]:
import math
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import ipywidgets as widgets
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec

camera_coords = np.array(np.zeros([2,2,2]))
epipole_coords = np.array(np.zeros([2,2]))
lambda1 = 0
lambda2 = 0
x1 = 0
x2 = 0



def update_2d_plots(fig,gs, camera_coords, epipole_coords, ep):
    
    #fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    #axs = [fig.add_subplot(1, 3, 2), fig.add_subplot(1, 3, 3)]
    axs = [fig.add_subplot(gs[0,3]),fig.add_subplot(gs[1,3])]
    # Clear existing plots
    axs[0].cla()
    axs[1].cla()
    
    # Update and configure the first plot (Camera 1 View)
    axs[0].set_title('Camera 1 View')
    axs[0].set_xlim(-1, 1)
    axs[0].set_ylim(-0.5, 0.5)
    rect1 = patches.Rectangle((-0.6, -0.4), 1.2, 0.8, color=(0, 1, 1, 0.2))  # Cyan color
    axs[0].add_patch(rect1)
    for i in range(0,1):
        axs[0].scatter(camera_coords[i][0][0], camera_coords[i][0][1], color='magenta')
        axs[0].text(camera_coords[i][0][0], camera_coords[i][0][1], f"x{i}", color='black')
        if ep:
            axs[0].scatter(epipole_coords[0][0], epipole_coords[0][1], color='black', marker='x')
            axs[0].plot([camera_coords[i][0][0], epipole_coords[0][0]], [camera_coords[i][0][1], epipole_coords[0][1]], color='green')


        # Update and configure the second plot (Camera 2 View)
    axs[1].set_title('Camera 2 View')
    axs[1].set_xlim(-1, 1)
    axs[1].set_ylim(-0.5, 0.5)
    rect2 = patches.Rectangle((-0.6, -0.4), 1.2, 0.8, color=(1, 1, 0, 0.2))  # Yellow color
    axs[1].add_patch(rect2)
    for i in range(0,1):
        axs[1].scatter(camera_coords[i][1][0], camera_coords[i][1][1], color='green')
        axs[1].text(camera_coords[i][1][0], camera_coords[i][1][1], f"x{i}\'", color='black')
        
        if ep:
            axs[1].plot([camera_coords[i][1][0], epipole_coords[1][0]], [camera_coords[i][1][1], epipole_coords[1][1]], color='magenta')
            axs[1].scatter(epipole_coords[1][0], epipole_coords[1][1], color='black', marker='x')
    # Redraw the plots
    plt.draw()


# Define a function to update the plot with both elevation and azimuth angles
def update_plot(elev_angle, azim_angle, roll_angle, FL2, Alpha, Beta, Gamma, tx, ty, tz, ep):
    # Create a new matplotlib figure and axis
    fig = plt.figure(figsize=(20, 6))
    gs = GridSpec(2, 4, figure=fig)
    #fig = plt.figure(figsize=(30, 10))
    #ax = fig.add_subplot(111, projection='3d')
    ax = fig.add_subplot(gs[0:4,:], projection='3d')
    # Set axes labels and limits
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_xlim([-2, 2])
    ax.set_ylim([-2, 2])
    ax.set_zlim([0, 4])
    global E_mat
    global rot


    # Second camera array
    K=np.array([[1,0,0,0],
               [0,1,0,0],
               [0,0,1,0],
               [0,0,0,1]])
    
    T = np.array([[1,0,0,tx],
                   [0,1,0,ty],
                   [0,0,1,tz],
                   [0,0,0,1]])
    
    # Individual Euler angle matrices
    alphaRot = np.array([[1,0,0,0],
       [0,math.cos(math.pi*Alpha/180),-math.sin(math.pi*Alpha/180),0],
       [0,math.sin(math.pi*Alpha/180),math.cos(math.pi*Alpha/180),0],
       [0,0,0,1]])
    betaRot = np.array([[math.cos(math.pi*Beta/180),0,math.sin(math.pi*Beta/180),0],
       [0,1,0,0],
       [-math.sin(math.pi*Beta/180),0,math.cos(math.pi*Beta/180),0],
       [0,0,0,1]])
    gammaRot = np.array([
       [math.cos(math.pi*Gamma/180),-math.sin(math.pi*Gamma/180),0,0],
       [math.sin(math.pi*Gamma/180),math.cos(math.pi*Gamma/180),0,0],
        [0,0,1,0],
       [0,0,0,1]])
    # Full rotation matrix but keep in mind that changing the order will change the rotation.
    rot = alphaRot @ betaRot @ gammaRot
    
    # Camera two focal length only.
    K_FL = ([[FL2,0,0,0],
       [0,FL2,0,0],
       [0,0,FL2,0],
       [0,0,0,1]])
    
    '''Special matrix for the applying the focal length to the z-axis only 
    This is used to move the image sensor with the focal length but not resize the sensor
    '''
    K_plane = ([[1,0,0,0],
               [0,1,0,0],
               [0,0 ,FL2,0],
               [0,0 ,0,1]])
    
    '''K_NF is the camera two matrix but without the focal length
    This is to all the red, green and blue axes for camera two 
    to be the same size as for camera one. So this matrix is to help 
    with the visualisation only'''
    K_NF = K @ T @ rot 
    
    '''K_z is for the visualisation only. It allows the camera two frame to be shown in the correct
    position without re-sizing the frame. Note, as we are only affecting the z-axis, ordering matters here.
    You must do the rotation and translation first and only then extend the z-axis or otherwise you will rotate
    and translate what you did to the z-axis and point it in another direction'''
   
    K_z = K @ T  @ rot @ K_plane 
   
    '''This is the full camera two matrix (relative to camera one). The focal length is in multiples 
    of the first camera focal length. Hence the first camera focal lenght is fixed at 1 and therefore all 
    coordinates are in units of the focal length of camera one'''
    
    K = K  @ T  @  rot @  K_FL  
    
       
    # Plotting the axes for the two cameras
    axes = np.array([[[-.1, 0, 0],[.1, 0, 0]],
            [[0, -.1, 0], [0, .1, 0]],
            [[0, 0, 0], [0, 0, 0.5]]])
               
       
    axes_cam_2 = axes.reshape(6,3)
    axes_cam_2 = np.hstack([axes_cam_2, np.ones((6, 1))])
    axes_cam_2 = K_NF @ axes_cam_2.transpose()
    axes_cam_2 = axes_cam_2.transpose() 
    # Remove the last column
    axes_cam_2 = axes_cam_2[:, :-1]
    axes_cam_2 = axes_cam_2.reshape(3,2,3)
    colors = ['r', 'g', 'b']  # Colors for each axis
    for i in range(0, 3):
        ax.plot([axes_cam_2[i][0][0], axes_cam_2[i][1][0]],  # X coordinates
            [axes_cam_2[i][0][1], axes_cam_2[i][1][1]],  # Y coordinates
            [axes_cam_2[i][0][2], axes_cam_2[i][1][2]],  # Z coordinates
            color=colors[i]) 
        
        ax.plot([axes[i][0][0], axes[i][1][0]],  # X coordinates
            [axes[i][0][1], axes[i][1][1]],  # Y coordinates
            [axes[i][0][2], axes[i][1][2]],  # Z coordinates
            color=colors[i])   
    
    intersection_point = np.zeros([2,3])
    cam_1_coord = np.zeros([2,2])
    cam_2_coord = np.zeros([2,2])
    intersection_point_imageP2 = np.zeros([2,4])
    points = np.zeros([2,3,3])
    world_coord = np.array([[0, 0.5, 3],
                              [-0.5, 0, 3]])
    for i in range(0,1):
        # adding the world coordinate point
        
        ax.scatter(*world_coord[i], color='black')
        ax.text(world_coord[i][0], world_coord[i][1], world_coord[i][2], f"X{i}", color='black')

        # Drawing a line from the origin to the  World coordinate point
        ax.plot([0, world_coord[i][0]], [0, world_coord[i][1]], [0, world_coord[i][2]], color='magenta')

        # Creating a plane normal to the y-axis centered at (0, 1, 0)
        x = np.linspace(-.6, .6, 10)
        y = np.linspace(-.4, .4, 10)
        X, Y = np.meshgrid(x, y)
        Z = np.ones_like(X)  # Plane centered at Z=focal length
        image_plane1 = np.array([X,Y,Z, np.ones_like(X)])



        camera_2_center = K_NF @ np.array([0,0,0,1])
        # Drawing a line from the camera 2 center to the point
        ax.plot([camera_2_center[0], world_coord[i][0]], [camera_2_center[1], world_coord[i][1]], [camera_2_center[2], world_coord[i][2]], color='green')

        # Adding the plane with transparency
        if i == 0:
            ax.plot_surface(image_plane1[0], image_plane1[1], image_plane1[2], color='cyan', alpha=0.2)

            # This reshapes image_plane1 for matrix multiplication by our camera 2 matrix
            image_plane2 = K_z @ image_plane1.reshape(4,-1) 

            # Reshaping back to original shape
            image_plane2 = image_plane2.reshape(image_plane1.shape) 
            ax.plot_surface(image_plane2[0], image_plane2[1], image_plane2[2], color='yellow', alpha=0.2)

        # The intersection point where the magenta line intersects the image_plane1 Z = 1
        intersection_point[i] = (world_coord[i][0]/world_coord[i][2], world_coord[i][1]/world_coord[i][2], world_coord[i][2]/world_coord[i][2])
        cam_1_coord[i] = np.array(intersection_point[i][:2])


        
        ax.scatter(*intersection_point[i], color='magenta')
        
        world_hom = np.array([world_coord[i][0],world_coord[i][1],world_coord[i][2],1])
        try:
            K_inv = np.linalg.inv(K)
        except np.linalg.LinAlgError:
            print("The matrix is not invertible.")

        temp_world = K_inv @ world_hom

        intersection_point_imageP2[i] = np.array([FL2*temp_world[0]/temp_world[2], 
                                      FL2*temp_world[1]/temp_world[2], 
                                      FL2*temp_world[2]/temp_world[2],1])

        x2 = intersection_point_imageP2[i][:3]
        cam_2_coord[i] = intersection_point_imageP2[i][:2]

        intersection_point_imageP2[i] = K_NF @ intersection_point_imageP2[i]
        pt = (intersection_point_imageP2[i][0],intersection_point_imageP2[i][1],intersection_point_imageP2[i][2])
        ax.scatter(*pt, color='green')

        # draw line between camera centers
        ax.plot([0, camera_2_center[0]], [0, camera_2_center[1]], [0, camera_2_center[2]], color='cyan')

        points[i] = np.array([[0, 0, 0],  # Origin - camera 1 center
                           [world_coord[i][0], world_coord[i][1], world_coord[i][2]],  # World coordinate
                           [camera_2_center[0], camera_2_center[1], camera_2_center[2]]])  # Camera 2 center

        # Shade in the Epipolar plane
        epipoloar_plane = Poly3DCollection([points[i]])
        epipoloar_plane.set_color('grey')
        epipoloar_plane.set_alpha(0.2)  # Adjust transparency here
        ax.add_collection3d(epipoloar_plane)
        
        if i == 0:
            # Show the epipole for camera 1
            cam_1_epipole = (camera_2_center[0]/camera_2_center[2], 
                             camera_2_center[1]/camera_2_center[2], 
                             camera_2_center[2]/camera_2_center[2])
            ax.scatter(*cam_1_epipole, color='black', marker='x')
            epipole_coords[0] = np.array([cam_1_epipole[0], cam_1_epipole[1]])

            # Show the epipole for camera 2
            cam_2_view_origin = K_inv @ np.array([0,0,0,1])
            cam_2_epipole = (cam_2_view_origin[0]/cam_2_view_origin[2], 
                             cam_2_view_origin[1]/cam_2_view_origin[2],
                             cam_2_view_origin[2]/cam_2_view_origin[2], 1)
            epipole_coords[1] = np.array([cam_2_epipole[0], cam_2_epipole[1]])
            cam_2_epipole = K @ cam_2_epipole
            ax.scatter(*cam_2_epipole[:3], color='black', marker='x')

            # Adjust view
            ax.view_init(elev=elev_angle, azim=azim_angle, roll=roll_angle)

        #show view in camera 1    
        cam_1_coord[i] = np.array([world_coord[i][0]/world_coord[i][2], world_coord[i][1]/world_coord[i][2]])


        camera_coords[i][0] = cam_1_coord[i]
        camera_coords[i][1] = cam_2_coord[i]

    
    
    update_2d_plots(fig,gs, camera_coords, epipole_coords, ep)
    
    lambda1 = world_coord[0][2]#math.sqrt((world_coord[0]**2)+(world_coord[1]**2)+(world_coord[2]**2))
    lambda2 = FL2*temp_world[2]# math.sqrt((world_coord[0]-camera_2_center[0])**2+(world_coord[1]-camera_2_center[1])**2+(world_coord[2]-camera_2_center[2])**2)
    print(f'lambda1:{lambda1}')
    print(f'lambda2:{lambda2}')
    
    x1 = np.append(cam_1_coord[0], 1)
    x2 = np.append(camera_coords[0][1], 1)
    print(f'x1:{x1}')
    #np.append(cam_2_coord, 1)
    print(f'x2:{x2}')
    Tx = np.array([[0, -tz, ty],
                   [tz, 0, -tx],
                   [-ty, tx, 0]])
    
    R = np.array(rot[:3,:3])
    print(f'x1TxRx2:{ x1 @ Tx @ R @ x2}')
    print(f'lambda1*x1:{lambda1*x1}')
    R_inv = np.linalg.inv(R)
    print(f'R(lambda1*x1)+T:{R_inv @ (lambda1*x1 -np.array([tx, ty,tz])) }' )
    print(f'lambda2*x2:{lambda2*x2}')
    
    E_mat= Tx @ R
    
    
    print(f'Determinant of E=TxR: {np.linalg.det(E_mat)}')
    # Show the plot
    plt.show()
    



elev_slider = widgets.IntSlider(min=-180, max=180, step=1, value=0, description='Elevation')
azim_slider = widgets.IntSlider(min=-180, max=180, step=1, value=90, description='Azimuth')
roll_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-90, description='Roll')



FL_slider2 = widgets.FloatSlider(min=0.1, max=3, step=0.1, value=1.0, description='Cam 2 Focal')

alpha_slider = widgets.IntSlider(min=-180, max=180, step=1, value=0, description='Cam2 Alpha')
beta_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-150, description='Cam2 Beta')
gamma_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-70, description='Cam2 Gamma')


tx_slider = widgets.FloatSlider(min=-2.0, max=2.0, step=0.1, value=1, description='Tx')
ty_slider = widgets.FloatSlider(min=-2.0, max=2.0, step=0.1, value=1, description='Ty')
tz_slider = widgets.FloatSlider(min=0.0, max=5.0, step=0.1, value=4, description='Tz')

# Group sliders into two columns
left_box = widgets.VBox([elev_slider, azim_slider, roll_slider, Xw_slider, Yw_slider, Zw_slider ])
right_box = widgets.VBox([ FL_slider2, alpha_slider, beta_slider, gamma_slider, tx_slider, ty_slider, tz_slider])

epipoles_checkbox = widgets.Checkbox(value=False, description='Show Epipoles in 2D',disabled=False)

# Combine the two columns into a single horizontal layout
ui = widgets.HBox([left_box,  right_box])


# Interactive widget
out = widgets.interactive_output(update_plot, {'elev_angle': elev_slider, 'azim_angle': azim_slider, 
                                               'roll_angle': roll_slider, 
                                                'FL2': FL_slider2, 
                                               'Alpha': alpha_slider, 'Beta': beta_slider, 'Gamma': gamma_slider,
                                              'tx': tx_slider, 'ty': ty_slider, 'tz': tz_slider, 'ep': epipoles_checkbox})

sliders_box = widgets.VBox([elev_slider, azim_slider, roll_slider, 
                            FL_slider2, alpha_slider, beta_slider, 
                            gamma_slider, tx_slider, ty_slider, tz_slider, epipoles_checkbox])
ui = widgets.HBox([sliders_box, out])


# Display the UI and the output widget
display(ui)

In [622]:
import math
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import ipywidgets as widgets
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec

camera_coords = np.array(np.zeros([8,2,2]))
epipole_coords = np.array(np.zeros([2,2]))
lambda1 = 0
lambda2 = 0
x1 = 0
x2 = 0
num_points = 8



def update_2d_plots(fig,gs, camera_coords, epipole_coords, ep):
    
    #fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    #axs = [fig.add_subplot(1, 3, 2), fig.add_subplot(1, 3, 3)]
    axs = [fig.add_subplot(gs[0,3]),fig.add_subplot(gs[1,3])]
    # Clear existing plots
    axs[0].cla()
    axs[1].cla()
    
    # Update and configure the first plot (Camera 1 View)
    axs[0].set_title('Camera 1 View')
    axs[0].set_xlim(-1, 1)
    axs[0].set_ylim(-0.5, 0.5)
    rect1 = patches.Rectangle((-0.6, -0.4), 1.2, 0.8, color=(0, 1, 1, 0.2))  # Cyan color
    axs[0].add_patch(rect1)
    for i in range(0,num_points):
        axs[0].scatter(camera_coords[i][0][0], camera_coords[i][0][1], color='magenta')
        axs[0].text(camera_coords[i][0][0], camera_coords[i][0][1], f"x{i}", color='black')
        if ep:
            axs[0].scatter(epipole_coords[0][0], epipole_coords[0][1], color='black', marker='x')
            axs[0].plot([camera_coords[i][0][0], epipole_coords[0][0]], [camera_coords[i][0][1], epipole_coords[0][1]], color='green')


        # Update and configure the second plot (Camera 2 View)
    axs[1].set_title('Camera 2 View')
    axs[1].set_xlim(-1, 1)
    axs[1].set_ylim(-0.5, 0.5)
    rect2 = patches.Rectangle((-0.6, -0.4), 1.2, 0.8, color=(1, 1, 0, 0.2))  # Yellow color
    axs[1].add_patch(rect2)
    for i in range(0,num_points):
        axs[1].scatter(camera_coords[i][1][0], camera_coords[i][1][1], color='green')
        axs[1].text(camera_coords[i][1][0], camera_coords[i][1][1], f"x{i}\'", color='black')
        
        if ep:
            axs[1].plot([camera_coords[i][1][0], epipole_coords[1][0]], [camera_coords[i][1][1], epipole_coords[1][1]], color='magenta')
            axs[1].scatter(epipole_coords[1][0], epipole_coords[1][1], color='black', marker='x')
    # Redraw the plots
    plt.draw()


# Define a function to update the plot with both elevation and azimuth angles
def update_plot(elev_angle, azim_angle, roll_angle, FL2, Alpha, Beta, Gamma, tx, ty, tz, ep):
    # Create a new matplotlib figure and axis
    fig = plt.figure(figsize=(20, 6))
    gs = GridSpec(2, 4, figure=fig)
    #fig = plt.figure(figsize=(30, 10))
    #ax = fig.add_subplot(111, projection='3d')
    ax = fig.add_subplot(gs[0:4,:], projection='3d')
    # Set axes labels and limits
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_xlim([-2, 2])
    ax.set_ylim([-2, 2])
    ax.set_zlim([0, 4])
    global E_mat
    global rot
    global T
    global world_coord


    # Second camera array
    K=np.array([[1,0,0,0],
               [0,1,0,0],
               [0,0,1,0],
               [0,0,0,1]])
    
    T = np.array([[1,0,0,tx],
                   [0,1,0,ty],
                   [0,0,1,tz],
                   [0,0,0,1]])
    
    # Individual Euler angle matrices
    alphaRot = np.array([[1,0,0,0],
       [0,math.cos(math.pi*Alpha/180),-math.sin(math.pi*Alpha/180),0],
       [0,math.sin(math.pi*Alpha/180),math.cos(math.pi*Alpha/180),0],
       [0,0,0,1]])
    betaRot = np.array([[math.cos(math.pi*Beta/180),0,math.sin(math.pi*Beta/180),0],
       [0,1,0,0],
       [-math.sin(math.pi*Beta/180),0,math.cos(math.pi*Beta/180),0],
       [0,0,0,1]])
    gammaRot = np.array([
       [math.cos(math.pi*Gamma/180),-math.sin(math.pi*Gamma/180),0,0],
       [math.sin(math.pi*Gamma/180),math.cos(math.pi*Gamma/180),0,0],
        [0,0,1,0],
       [0,0,0,1]])
    # Full rotation matrix but keep in mind that changing the order will change the rotation.
    rot = alphaRot @ betaRot @ gammaRot
    
    # Camera two focal length only.
    K_FL = ([[FL2,0,0,0],
       [0,FL2,0,0],
       [0,0,FL2,0],
       [0,0,0,1]])
    
    '''Special matrix for the applying the focal length to the z-axis only 
    This is used to move the image sensor with the focal length but not resize the sensor
    '''
    K_plane = ([[1,0,0,0],
               [0,1,0,0],
               [0,0 ,FL2,0],
               [0,0 ,0,1]])
    
    '''K_NF is the camera two matrix but without the focal length
    This is to all the red, green and blue axes for camera two 
    to be the same size as for camera one. So this matrix is to help 
    with the visualisation only'''
    K_NF = K @ T @ rot 
    
    '''K_z is for the visualisation only. It allows the camera two frame to be shown in the correct
    position without re-sizing the frame. Note, as we are only affecting the z-axis, ordering matters here.
    You must do the rotation and translation first and only then extend the z-axis or otherwise you will rotate
    and translate what you did to the z-axis and point it in another direction'''
   
    K_z = K @ T  @ rot @ K_plane 
   
    '''This is the full camera two matrix (relative to camera one). The focal length is in multiples 
    of the first camera focal length. Hence the first camera focal lenght is fixed at 1 and therefore all 
    coordinates are in units of the focal length of camera one'''
    
    K = K  @ T  @  rot @  K_FL  
    
       
    # Plotting the axes for the two cameras
    axes = np.array([[[-.1, 0, 0],[.1, 0, 0]],
            [[0, -.1, 0], [0, .1, 0]],
            [[0, 0, 0], [0, 0, 0.5]]])
               
       
    axes_cam_2 = axes.reshape(6,3)
    axes_cam_2 = np.hstack([axes_cam_2, np.ones((6, 1))])
    axes_cam_2 = K_NF @ axes_cam_2.transpose()
    axes_cam_2 = axes_cam_2.transpose() 
    # Remove the last column
    axes_cam_2 = axes_cam_2[:, :-1]
    axes_cam_2 = axes_cam_2.reshape(3,2,3)
    colors = ['r', 'g', 'b']  # Colors for each axis
    for i in range(0, 3):
        ax.plot([axes_cam_2[i][0][0], axes_cam_2[i][1][0]],  # X coordinates
            [axes_cam_2[i][0][1], axes_cam_2[i][1][1]],  # Y coordinates
            [axes_cam_2[i][0][2], axes_cam_2[i][1][2]],  # Z coordinates
            color=colors[i]) 
        
        ax.plot([axes[i][0][0], axes[i][1][0]],  # X coordinates
            [axes[i][0][1], axes[i][1][1]],  # Y coordinates
            [axes[i][0][2], axes[i][1][2]],  # Z coordinates
            color=colors[i])   
    
    intersection_point = np.zeros([8,3])
    cam_1_coord = np.zeros([8,2])
    cam_2_coord = np.zeros([8,2])
    intersection_point_imageP2 = np.zeros([8,4])
    points = np.zeros([8,3,3])
    world_coord = np.array([[-0.5, 0.5, 1.3],
                            [-0.5, 0.2, 2.5],
                            [-0.5, 0.3, 1.5],
                            [-0.2,0.4,2.4],
                            [-0.5,0.5,2.7],
                             [0.2,0.7,1.9],
                             [-0.3,0.7,2.7],
                             [-0.2,0.8,2.3]],) 
    
    
    
    for i in range(0,num_points):
        # adding the world coordinate point
        
        ax.scatter(*world_coord[i], color='black')
        ax.text(world_coord[i][0], world_coord[i][1], world_coord[i][2], f"X{i}", color='black')

        # Drawing a line from the origin to the  World coordinate point
        ax.plot([0, world_coord[i][0]], [0, world_coord[i][1]], [0, world_coord[i][2]], color='magenta')

        # Creating a plane normal to the y-axis centered at (0, 1, 0)
        x = np.linspace(-.6, .6, 10)
        y = np.linspace(-.4, .4, 10)
        X, Y = np.meshgrid(x, y)
        Z = np.ones_like(X)  # Plane centered at Z=focal length
        image_plane1 = np.array([X,Y,Z, np.ones_like(X)])



        camera_2_center = K_NF @ np.array([0,0,0,1])
        # Drawing a line from the camera 2 center to the point
        ax.plot([camera_2_center[0], world_coord[i][0]], [camera_2_center[1], world_coord[i][1]], [camera_2_center[2], world_coord[i][2]], color='green')

        # Adding the plane with transparency
        if i == 0:
            ax.plot_surface(image_plane1[0], image_plane1[1], image_plane1[2], color='cyan', alpha=0.2)

            # This reshapes image_plane1 for matrix multiplication by our camera 2 matrix
            image_plane2 = K_z @ image_plane1.reshape(4,-1) 

            # Reshaping back to original shape
            image_plane2 = image_plane2.reshape(image_plane1.shape) 
            ax.plot_surface(image_plane2[0], image_plane2[1], image_plane2[2], color='yellow', alpha=0.2)

        # The intersection point where the magenta line intersects the image_plane1 Z = 1
        intersection_point[i] = (world_coord[i][0]/world_coord[i][2], world_coord[i][1]/world_coord[i][2], world_coord[i][2]/world_coord[i][2])
        cam_1_coord[i] = np.array(intersection_point[i][:2])


        
        ax.scatter(*intersection_point[i], color='magenta')
        
        world_hom = np.array([world_coord[i][0],world_coord[i][1],world_coord[i][2],1])
        try:
            K_inv = np.linalg.inv(K)
        except np.linalg.LinAlgError:
            print("The matrix is not invertible.")

        temp_world = K_inv @ world_hom
        if i == 0:
            temp_world0=temp_world
        intersection_point_imageP2[i] = np.array([FL2*temp_world[0]/temp_world[2], 
                                      FL2*temp_world[1]/temp_world[2], 
                                      FL2*temp_world[2]/temp_world[2],1])

        x2 = intersection_point_imageP2[i][:3]
        cam_2_coord[i] = intersection_point_imageP2[i][:2]

        intersection_point_imageP2[i] = K_NF @ intersection_point_imageP2[i]
        pt = (intersection_point_imageP2[i][0],intersection_point_imageP2[i][1],intersection_point_imageP2[i][2])
        ax.scatter(*pt, color='green')

        # draw line between camera centers
        ax.plot([0, camera_2_center[0]], [0, camera_2_center[1]], [0, camera_2_center[2]], color='cyan')

        points[i] = np.array([[0, 0, 0],  # Origin - camera 1 center
                           [world_coord[i][0], world_coord[i][1], world_coord[i][2]],  # World coordinate
                           [camera_2_center[0], camera_2_center[1], camera_2_center[2]]])  # Camera 2 center

        # Shade in the Epipolar plane
        epipoloar_plane = Poly3DCollection([points[i]])
        epipoloar_plane.set_color('grey')
        epipoloar_plane.set_alpha(0.2)  # Adjust transparency here
        ax.add_collection3d(epipoloar_plane)
        
        if i == 0:
            # Show the epipole for camera 1
            cam_1_epipole = (camera_2_center[0]/camera_2_center[2], 
                             camera_2_center[1]/camera_2_center[2], 
                             camera_2_center[2]/camera_2_center[2])
            ax.scatter(*cam_1_epipole, color='black', marker='x')
            epipole_coords[0] = np.array([cam_1_epipole[0], cam_1_epipole[1]])

            # Show the epipole for camera 2
            cam_2_view_origin = K_inv @ np.array([0,0,0,1])
            cam_2_epipole = (cam_2_view_origin[0]/cam_2_view_origin[2], 
                             cam_2_view_origin[1]/cam_2_view_origin[2],
                             cam_2_view_origin[2]/cam_2_view_origin[2], 1)
            epipole_coords[1] = np.array([cam_2_epipole[0], cam_2_epipole[1]])
            cam_2_epipole = K @ cam_2_epipole
            ax.scatter(*cam_2_epipole[:3], color='black', marker='x')

            # Adjust view
            ax.view_init(elev=elev_angle, azim=azim_angle, roll=roll_angle)

        #show view in camera 1    
        cam_1_coord[i] = np.array([world_coord[i][0]/world_coord[i][2], world_coord[i][1]/world_coord[i][2]])


        camera_coords[i][0] = cam_1_coord[i]
        camera_coords[i][1] = cam_2_coord[i]

    
    
    update_2d_plots(fig,gs, camera_coords, epipole_coords, ep)
    
    lambda1 = world_coord[0][2]#math.sqrt((world_coord[0]**2)+(world_coord[1]**2)+(world_coord[2]**2))
    lambda2 = FL2*temp_world0[2]# math.sqrt((world_coord[0]-camera_2_center[0])**2+(world_coord[1]-camera_2_center[1])**2+(world_coord[2]-camera_2_center[2])**2)
    print(f'lambda1:{lambda1}')
    print(f'lambda2:{lambda2}')
    
    x0 = np.append(camera_coords[0][0], 1)
    
    x0p = np.append(camera_coords[0][1], 1)
    print(f'x0:{x0}')
    #np.append(cam_2_coord, 1)
    print(f'x0\':{x0p}')
    Tx = np.array([[0, -tz, ty],
                   [tz, 0, -tx],
                   [-ty, tx, 0]])
    
    R = np.array(rot[:3,:3])
    print(f'x0TxRx0\':{ x0 @ Tx @ R @ x0p}')
    print(f'lambda1*x1:{lambda1*x0}')
    R_inv = np.linalg.inv(R)
    print(f'R(lambda1*x0)+T:{R_inv @ (lambda1*x0 -np.array([tx, ty,tz])) }' )
    print(f'lambda2*x0\':{lambda2*x0p}')
    
    E_mat= Tx @ R
    
    
    print(f'Determinant of E=TxR: {np.linalg.det(E_mat)}')
    # Show the plot
    plt.show()
    



elev_slider = widgets.IntSlider(min=-180, max=180, step=1, value=0, description='Elevation')
azim_slider = widgets.IntSlider(min=-180, max=180, step=1, value=90, description='Azimuth')
roll_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-90, description='Roll')



FL_slider2 = widgets.FloatSlider(min=0.1, max=3, step=0.1, value=1.0, description='Cam 2 Focal')

alpha_slider = widgets.IntSlider(min=-180, max=180, step=1, value=0, description='Cam2 Alpha')
beta_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-150, description='Cam2 Beta')
gamma_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-70, description='Cam2 Gamma')


tx_slider = widgets.FloatSlider(min=-2.0, max=2.0, step=0.1, value=1, description='Tx')
ty_slider = widgets.FloatSlider(min=-2.0, max=2.0, step=0.1, value=1, description='Ty')
tz_slider = widgets.FloatSlider(min=0.0, max=5.0, step=0.1, value=4, description='Tz')

# Group sliders into two columns
left_box = widgets.VBox([elev_slider, azim_slider, roll_slider, Xw_slider, Yw_slider, Zw_slider ])
right_box = widgets.VBox([ FL_slider2, alpha_slider, beta_slider, gamma_slider, tx_slider, ty_slider, tz_slider])

epipoles_checkbox = widgets.Checkbox(value=False, description='Show Epipoles in 2D',disabled=False)

# Combine the two columns into a single horizontal layout
ui = widgets.HBox([left_box,  right_box])


# Interactive widget
out = widgets.interactive_output(update_plot, {'elev_angle': elev_slider, 'azim_angle': azim_slider, 
                                               'roll_angle': roll_slider, 
                                                'FL2': FL_slider2, 
                                               'Alpha': alpha_slider, 'Beta': beta_slider, 'Gamma': gamma_slider,
                                              'tx': tx_slider, 'ty': ty_slider, 'tz': tz_slider, 'ep': epipoles_checkbox})

sliders_box = widgets.VBox([elev_slider, azim_slider, roll_slider, 
                            FL_slider2, alpha_slider, beta_slider, 
                            gamma_slider, tx_slider, ty_slider, tz_slider, epipoles_checkbox])
ui = widgets.HBox([sliders_box, out])


# Display the UI and the output widget
display(ui)

In [619]:
print(camera_coords.shape)
print(T)
print(rot)
print(rot @ T)
rotTInv = np.linalg.inv(rot @ T)
print(rotTInv)
rotinvTminus = rot.T @ T
print(rotinvTminus)

'''X = [1,1,1,1]
print(rotTInv @ X)
print(rotinvTminus @ X)
print(rot)
print(rot.T)'''

(8, 2, 2)
[[1. 0. 0. 1.]
 [0. 1. 0. 1.]
 [0. 0. 1. 4.]
 [0. 0. 0. 1.]]
[[-0.296 -0.814 -0.5    0.   ]
 [-0.94   0.342  0.     0.   ]
 [ 0.171  0.47  -0.866  0.   ]
 [ 0.     0.     0.     1.   ]]
[[-0.296 -0.814 -0.5   -3.11 ]
 [-0.94   0.342  0.    -0.598]
 [ 0.171  0.47  -0.866 -2.823]
 [ 0.     0.     0.     1.   ]]
[[-0.296 -0.94   0.171 -1.   ]
 [-0.814  0.342  0.47  -1.   ]
 [-0.5    0.    -0.866 -4.   ]
 [ 0.     0.     0.     1.   ]]
[[-0.296 -0.94   0.171 -0.552]
 [-0.814  0.342  0.47   1.408]
 [-0.5    0.    -0.866 -3.964]
 [ 0.     0.     0.     1.   ]]


'X = [1,1,1,1]\nprint(rotTInv @ X)\nprint(rotinvTminus @ X)\nprint(rot)\nprint(rot.T)'

## The Eight Point Algorithm

So we start with the epipolar constraint.
\begin{equation}
    x_2^{\top}Ex_1=0
\end{equation}

This should hold for any matching points $x_1$ and $x_2$ in two image views.

If we have enough points, we will use eight, then we should be able to recover the unknown matrix $E$.

Strictly speaking we can get away with seven points but eight will give us a unique solution (up to scale).

This assumes the eight points meet certain criteria and have no noise. More on this later.



$E$ is a $3\times3$ matrix as follows.
\begin{equation}
	E= \begin{bmatrix}
e_{11} & e_{12} & e_{13}\\
e_{21} & e_{22} & e_{23}\\
e_{31} & e_{32} & e_{33}\\
 \end{bmatrix}
\end{equation}
    

We can stack this matrix into a single vector $\in \mathbb{R}^9$

which we will call $e_s$

$$e_s= \begin{bmatrix}
e_{11}\\
e_{21}\\
e_{31}\\
e_{12}\\
e_{22}\\
e_{32}\\
e_{13}\\
e_{23}\\
e_{33}
\end{bmatrix} $$
 
For each set of point correspondences in homogeneous coordinates we will get one linear equation in the unknown entries of $E$.

For example if $x = (x,y,1)^{\top}$ and $x' = (x',y',1)^{\top}$ we have the linear equation

 \begin{equation}
	x'xe_{11}+x'ye_{12}+x'e_{13}+y'xe_{21}+y'ye_{22}+y'e_{23}+xe_{31}+ye_{32}+e_{33} = 0
\end{equation}

We can write this as $a^{\top}e_s=0$ which is equivalent to the epipolar constraint.


$$ \begin{bmatrix}
x'x & x'y & x'& y'x&y'y&y'&x&y&1
 \end{bmatrix} \begin{bmatrix}
e_{11}\\
e_{21}\\
e_{31}\\
e_{12}\\
e_{22}\\
e_{32}\\
e_{13}\\
e_{23}\\
e_{33}
\end{bmatrix}=0$$

 
     





 
We can put each of the equations for the eight point correspondances into a separate row of a matrix called $A$
 \begin{equation}
	\begin{bmatrix}
x_1'x_1 & x_1'y_1 & x_1'& y_1'x_1&y_1'y_1&y_1'&x_1&y_1&1\\
x_2'x_2 & x_2'y_2 & x_2'& y_2'x_2&y_2'y_2&y_2'&x_2&y_2&1\\
x_3'x_3 & x_3'y_3 & x_3'& y_3'x_3&y_3'y_3&y_3'&x_3&y_3&1\\
x_4'x_4 & x_4'y_4 & x_4'& y_4'x_4&y_4'y_4&y_4'&x_4&y_4&1\\
x_5'x_5 & x_5'y_5 & x_5'& y_5'x_5&y_5'y_5&y_5'&x_5&y_5&1\\
x_6'x_6 & x_6'y_6 & x_6'& y_6'x_6&y_6'y_6&y_6'&x_6&y_6&1\\
x_7'x_7 & x_7'y_7 & x_7'& y_7'x_7&y_7'y_7&y_7'&x_7&y_7&1\\
x_8'x_8 & x_8'y_8 & x_8'& y_8'x_8&y_8'y_8&y_8'&x_8&y_8&1\\
 \end{bmatrix} \begin{bmatrix}
e_{11}\\
e_{21}\\
e_{31}\\
e_{12}\\
e_{22}\\
e_{32}\\
e_{13}\\
e_{23}\\
e_{33} 
 \end{bmatrix} = 0
\end{equation}

  

In [660]:
c = camera_coords


A = np.array([[c[0,1,0]*c[0,0,0], c[0,1,0]*c[0,0,1], c[0,1,0], c[0,1,1]*c[0,0,0], c[0,1,1]*c[0,0,1], c[0,1,1], c[0,0,0], c[0,0,1], 1 ],
              [c[1,1,0]*c[1,0,0], c[1,1,0]*c[1,0,1], c[1,1,0], c[1,1,1]*c[1,0,0], c[1,1,1]*c[1,0,1], c[1,1,1], c[1,0,0], c[1,0,1], 1 ],
              [c[2,1,0]*c[2,0,0], c[2,1,0]*c[2,0,1], c[2,1,0], c[2,1,1]*c[2,0,0], c[2,1,1]*c[2,0,1], c[2,1,1], c[2,0,0], c[2,0,1], 1 ],
              [c[3,1,0]*c[3,0,0], c[3,1,0]*c[3,0,1], c[3,1,0], c[3,1,1]*c[3,0,0], c[3,1,1]*c[3,0,1], c[3,1,1], c[3,0,0], c[3,0,1], 1 ],
              [c[4,1,0]*c[4,0,0], c[4,1,0]*c[4,0,1], c[4,1,0], c[4,1,1]*c[4,0,0], c[4,1,1]*c[4,0,1], c[4,1,1], c[4,0,0], c[4,0,1], 1 ],
              [c[5,1,0]*c[5,0,0], c[5,1,0]*c[5,0,1], c[5,1,0], c[5,1,1]*c[5,0,0], c[5,1,1]*c[5,0,1], c[5,1,1], c[5,0,0], c[5,0,1], 1 ],
              [c[6,1,0]*c[6,0,0], c[6,1,0]*c[6,0,1], c[6,1,0], c[6,1,1]*c[6,0,0], c[6,1,1]*c[6,0,1], c[6,1,1], c[6,0,0], c[6,0,1], 1 ],
              [c[7,1,0]*c[7,0,0], c[7,1,0]*c[7,0,1], c[7,1,0], c[7,1,1]*c[7,0,0], c[7,1,1]*c[7,0,1], c[7,1,1], c[7,0,0], c[7,0,1], 1 ]])

U, S, Vh = np.linalg.svd(A.T @ A)
print(f'S:{S}')
Es = Vh[8,:]
Es = Es/Es[8]
E = Es.reshape(3,3)
#E = E.transpose()

print(f'Es:{Es}')
print(f'E:{E}')
print(f'ATe:{A @ Es}')



#In this code I am using the E matrix that I created from a known rotation and translation matrix
# For the eight point algorithm we must determine the E matrix
U, S, Vh = np.linalg.svd(E)
theta = np.pi / 2  # 90 degrees in radians
print(f'S:{S}')
negDetFlag = 1
if np.linalg.det(E) < 0.0:
    print('Negative Determinent')
    negDetFlag = -1

S[2] = 0

Rz = np.array([[np.cos(theta), -np.sin(theta),0],
                [np.sin(theta), np.cos(theta),0],
                [0, 0, 1]])
Rzneg = np.array([[np.cos(-theta), -np.sin(-theta),0],
                [np.sin(-theta), np.cos(-theta),0],
                 [0, 0, 1]])

Tx1 = U @ Rz @ np.diag(S) @ U.T
Tx2 = U @ Rzneg @ np.diag(S) @ U.T
T1 = np.array([Tx1[2, 1], Tx1[0, 2], Tx1[1, 0]])
T2 = np.array([Tx2[2, 1], Tx2[0, 2], Tx2[1, 0]])

print(f'T1: {T1}')
print(f'T2: {T2}')


np.set_printoptions(precision=3, suppress=True)
R1 = negDetFlag*(U @ Rz.T @ Vh)
R2 = negDetFlag*(U @ Rzneg.T @ Vh)
print(f'R1:{R1}')
print(f'R2:{R2}')
print(f'Actual Rotation: {rot[0:3,0:3]}')



print(f'determinant of U: {np.linalg.det(U)}')
print(f'determinant of Vh: {np.linalg.det(Vh)}')
print(f'determinant of E: {np.linalg.det(E)}')
print(f'determinant of S: {np.linalg.det(np.diag(S))}')
print(f'determinant of Rz: {np.linalg.det(Rz)}')
print(f'determinant of Rzneg: {np.linalg.det(Rzneg)}')
print(f'determinant of R2: {np.linalg.det(R2)}')
print(f'determinant of R1: {np.linalg.det(R1)}')
print(f'determinant of actual rotation: {np.linalg.det(rot[0:3,0:3])}')
X = 1.3*np.array([-0.3846, 0.3846, 1])
Xp = 3.08826868*np.array([0.1465, -0.0709, 1])
print(1.3*np.array([-0.3846, 0.3846, 1]))
print(3.08826868*np.array([0.1465, -0.0709, 1]))
print( rot[0:3,0:3].T @ (X - T[0:3,3]) ) 
print( rot[0:3,0:3] @ Xp + T[0:3,3] ) 

print( R1 @ X + T1/2)
print( R2 @ X + T2/2)


# How best to explain the recovered T. I guess simply that T it recovers the homogeneous coordinate T in camera one rather than the actual Translation.
# I also need to figure out the difference between recovery to a scalar versus recorvery to a projection.

S:[9.24  0.31  0.155 0.069 0.004 0.001 0.    0.    0.   ]
Es:[ 7.86  -2.712 -1.287 -1.796 -7.45   2.312 -1.732 -2.268  1.   ]
E:[[ 7.86  -2.712 -1.287]
 [-1.796 -7.45   2.312]
 [-1.732 -2.268  1.   ]]
ATe:[-0. -0. -0.  0.  0. -0. -0.  0.]
S:[8.485 8.485 0.   ]
Negative Determinent
T1: [ 1.104 -2.815  7.928]
T2: [-1.104  2.815 -7.928]
R1:[[ 0.235  0.878 -0.416]
 [ 0.97  -0.186  0.156]
 [ 0.06  -0.44  -0.896]]
R2:[[-0.296 -0.94   0.171]
 [-0.814  0.342  0.47 ]
 [-0.5    0.    -0.866]]
Actual Rotation: [[-0.296 -0.814 -0.5  ]
 [-0.94   0.342  0.   ]
 [ 0.171  0.47  -0.866]]
determinant of U: 0.9999999999999998
determinant of Vh: -1.0
determinant of E: -6.451319239206104e-10
determinant of S: 0.0
determinant of Rz: 1.0
determinant of Rzneg: 1.0
determinant of R2: 0.9999999999999996
determinant of R1: 0.9999999999999996
determinant of actual rotation: 1.0
[-0.5  0.5  1.3]
[ 0.452 -0.219  3.088]
[ 0.452 -0.219  3.088]
[-0.5  0.5  1.3]
[ 0.332 -1.783  2.55 ]
[-0.651  2.596 -4.84 ]


In [681]:
#playing around
np.set_printoptions(precision=12, suppress=True)
c = camera_coords

print(f'({c[0,0,0]},{c[0,0,1]}')
print(f'({c[0,1,0]},{c[0,1,1]}')
A = np.array([[c[0,0,0]*c[0,1,0], c[0,0,0]*c[0,1,1], c[0,0,0], c[0,0,1]*c[0,1,0], c[0,0,1]*c[0,1,1], c[0,0,1], c[0,1,0], c[0,1,1], 1 ],
              [c[1,0,0]*c[1,1,0], c[1,0,0]*c[1,1,1], c[1,0,0], c[1,0,1]*c[1,1,0], c[1,0,1]*c[1,1,1], c[1,0,1], c[1,1,0], c[1,1,1], 1 ],
              [c[2,0,0]*c[2,1,0], c[2,0,0]*c[2,1,1], c[2,0,0], c[2,0,1]*c[2,1,0], c[2,0,1]*c[2,1,1], c[2,0,1], c[2,1,0], c[2,1,1], 1 ],
              [c[3,0,0]*c[3,1,0], c[3,0,0]*c[3,1,1], c[3,0,0], c[3,0,1]*c[3,1,0], c[3,0,1]*c[3,1,1], c[3,0,1], c[3,1,0], c[3,1,1], 1 ],
              [c[4,0,0]*c[4,1,0], c[4,0,0]*c[4,1,1], c[4,0,0], c[4,0,1]*c[4,1,0], c[4,0,1]*c[4,1,1], c[4,0,1], c[4,1,0], c[4,1,1], 1 ],
              [c[5,0,0]*c[5,1,0], c[5,0,0]*c[5,1,1], c[5,0,0], c[5,0,1]*c[5,1,0], c[5,0,1]*c[5,1,1], c[5,0,1], c[5,1,0], c[5,1,1], 1 ],
              [c[6,0,0]*c[6,1,0], c[6,0,0]*c[6,1,1], c[6,0,0], c[6,0,1]*c[6,1,0], c[6,0,1]*c[6,1,1], c[6,0,1], c[6,1,0], c[6,1,1], 1 ],
              [c[7,0,0]*c[7,1,0], c[7,0,0]*c[7,1,1], c[7,0,0], c[7,0,1]*c[7,1,0], c[7,0,1]*c[7,1,1], c[7,0,1], c[7,1,0], c[7,1,1], 1 ]])

print(A.shape)
U, S, Vh = np.linalg.svd(A)
#print(U)
print(S)
#print(Vh)
#print("************")
#print(Vh[:,8])
Es = Vh[8,:]
print(Es)
Es = Es/Es[8]
E = Es.reshape(3,3)



print(Es)
print(f'Calculated E_Det: {np.linalg.det(E)}')
print(f'Calculated E: {E}')



#In this code I am using the E matrix that I created from a known rotation and translation matrix
# For the eight point algorithm we must determine the E matrix

Tx = skew(np.array([1,1,4]))
#print(Tx)
synE = Tx @ rot[0:3,0:3]
synE = synE/synE[2,2]
print(f'Synthetic E: {synE}')
print(f'Synthetic E_Det: {np.linalg.det(synE)}')
print(f' Dif synE-E:{synE-E}')
U, S, Vh = np.linalg.svd(synE)

theta = np.pi / 2  # 90 degrees in radians
print(f'S:{S}')
print(f'S[2]:{S[2]}')
negDetFlag = 1
if np.linalg.det(E) < 0.0:
    print('Negative Determinent')
    negDetFlag = -1
S[2] = 0
E = U @ np.diag(S) @ Vh
U, S, Vh = np.linalg.svd(E)



Rz = np.array([[np.cos(theta), -np.sin(theta),0],
                [np.sin(theta), np.cos(theta),0],
                [0, 0, 1]])
Rzneg = np.array([[np.cos(-theta), -np.sin(-theta),0],
                [np.sin(-theta), np.cos(-theta),0],
                 [0, 0, 1]])
Tx1 = U @ Rz @ np.diag(S) @ U.T
Tx2 = U @ Rzneg @ np.diag(S) @ U.T
T1 = np.array([Tx1[2, 1], Tx1[0, 2], Tx1[1, 0]])
T2 = np.array([Tx2[2, 1], Tx2[0, 2], Tx2[1, 0]])
print(f'Tx1: {Tx1}')
print(f'Tx2: {Tx2}')
print(f'T1: {T1}')
print(f'T2: {T2}')



R1 = negDetFlag*(U @ Rz.T @ Vh)
R2 = negDetFlag*(U @ Rzneg.T @ Vh)
print(f'R1:{R1}')
print(f'R2:{R2}')
print(f'Actual Rotation: {rot[0:3,0:3]}')
#print(f'Actual Rotation inv: {rotInv}')
print(S)
 
print(T2)
#T2 = 4.3*T2/0.701
print(T)
print(f'determinant of U: {np.linalg.det(U)}')
print(f'determinant of Vh: {np.linalg.det(Vh)}')
print(f'determinant of E: {np.linalg.det(E)}')
print(f'determinant of S: {np.linalg.det(np.diag(S))}')
print(f'determinant of Rz: {np.linalg.det(Rz)}')
print(f'determinant of Rzneg: {np.linalg.det(Rzneg)}')
print(f'determinant of R2: {np.linalg.det(R2)}')
print(f'determinant of R1: {np.linalg.det(R1)}')
print(f'determinant of actual rotation: {np.linalg.det(rot[0:3,0:3])}')

X = 1.3*np.array([-0.3846, 0.3846, 1])
Xp = 3.08826868*np.array([0.1465, -0.0709, 1])
print(1.3*np.array([-0.3846, 0.3846, 1]))
print(3.08826868*np.array([0.1465, -0.0709, 1]))
print( rot[0:3,0:3].T @ (X - T[0:3,3]) ) 

print( R1 @ Xp + T1/2)
print( R2 @ Xp + T2/2)
print( R1 @ X + T1/2)
print( R2 @ X + T2/2)
# How best to explain the recovered T. I guess simply that T it recovers the homogeneous coordinate T in camera one rather than the actual Translation.
# I also need to figure out the difference between recovery to a scalar versus recorvery to a projection.

(-0.3846153846153846,0.3846153846153846
(0.14649513239403944,-0.07088068323885628
(8, 9)
[3.039762948063 0.55649307714  0.393403515918 0.263331614303
 0.060835445213 0.038258743933 0.020225585776 0.002310344643]
[ 0.654963425801 -0.149705710485 -0.144337567297 -0.225967100428
 -0.620839505965 -0.188995766036 -0.107249081343  0.192636304113
  0.083333333333]
[ 7.859561109613 -1.796468525819 -1.732050807569 -2.711605205134
 -7.450074071581 -2.267949192431 -1.28698897612   2.31163564935
  1.            ]
Calculated E_Det: -3.4123543974788306e-13
Calculated E: [[ 7.859561109613 -1.796468525819 -1.732050807569]
 [-2.711605205134 -7.450074071581 -2.267949192431]
 [-1.28698897612   2.31163564935   1.            ]]
Synthetic E: [[ 7.859561109613 -1.796468525819 -1.732050807569]
 [-2.711605205134 -7.450074071581 -2.267949192431]
 [-1.28698897612   2.31163564935   1.            ]]
Synthetic E_Det: 7.041659016127529e-15
 Dif synE-E:[[-0.  0.  0.]
 [ 0. -0.  0.]
 [-0.  0.  0.]]
S:[8.485281374239 8

## Why Eight?
You will notice that there are nine unknowns in $E$ but as mentioned earlier we can only determine a unique solution up to a scale factor. So generally we set $e_{33}$ to 1 as we can set it to any value. But we must do this first before we start solving.

For $E$ to have a solution, $A$ must be of rank at most 8. 

If it is of rank = 8 then we will have a unique solution. $Ae_s = 0$ says that the vector $e_s$ is in the null space of $A$. So if $A$ had rank greater than 8 then there would be no null space and no solution. 

For less than 8 e.g. 7 there is a whole 2D plane of solutions and there are methods to determine a solution on this plane with the best essential matrix in $\mathcal{E}$ space.

## Noise
**If the data (point correspondences) are not exact (it won't be exact in the real world), the rank of $A$ may be greater than 8.**

9 is the full rank as there are 9 columns.

In this case we can find the least squares solution. 

This is also the case if we use more than 8 point correspondences.

Also note that even in the supposed full rank case we would need 9 correspondences to realise this. 

So in the case of noise and only 8 points, we will still only have rank 8, but our null space could be the wrong vector/line. 

    

In [218]:
#I can demonstrate that by adding noise I don't recover the Rotation and Translation.

## How do we find the Null space of A?
How do we find $e_s$?

Get the SVD of $A$.
\begin{equation}
	A = U\Sigma V^{\top}
\end{equation}

The solution is the column vector of $V$ that corresponds to the smallest singular value of $\Sigma$. 

Most programming software libraries (Matlab, Numpy) will order the singular values of $\Sigma$ in descending order. 

In that case the solution $e_s$ will be the final column of $V^{\top}$.

## Projecting $E$ onto the essential space ($\mathcal{E}$)

As we mentioned, calculating $E$ by this manner is unlikely to get us a matrix that obeys all the constraints of the essential space.

Instead we must project onto $\mathcal{E}$.

From Theorem: _Projection onto the essential space_ page 86 An invitation to 3D Vision by Ma, Kosecka, Soatto, Sastry.


Let $E$ be the calculated matrix $\in \mathbb{R}^{3\times3}$.

Perform an SVD on E such that
\begin{equation}
	E = U \begin{bmatrix}
\lambda_1 & 0 & 0\\
0 & \lambda_2 & 0\\
0 & 0 &\lambda_3
 \end{bmatrix}V^{\top}
\end{equation}

Where $\lambda_1 \geq \lambda_2 \geq \lambda_3$, i.e. the singular values are in descending order.
The closest essential matrix (we'll call it $E^*$) is given by 
 \begin{equation}
	E^* = U \begin{bmatrix}
	\sigma & 0 & 0\\
0 & \sigma & 0\\
0 & 0 &0
 \end{bmatrix}V^{\top}, \quad \text{with } \sigma = \frac{\lambda_1+\lambda_2}{2}
\end{equation}




  

## Two decompositions

From the theorem _Pose recovery from the Essential Matrix_ - page 84 An invitation to 3D Vision by Ma, Kosecka, Soatto, Sastry.

There are two relative poses $(R,T)$ with $R\in SO(3)$ and $T\in\mathbb{R}^3$ corresponding to an essential matrix $E\in\mathcal{E}$


For $E=U\Sigma V^{\top}$ we have:
\begin{equation}
	(T_{1\times},R_1) = (UR_{Z(+\frac{\pi}{2})}\Sigma U^{\top}, UR^{\top}_{Z(+\frac{\pi}{2})}V^{\top})
\end{equation}
\begin{equation}
	(T_{2\times},R_2) = (UR_{Z(-\frac{\pi}{2})}\Sigma U^{\top}, UR^{\top}_{Z(-\frac{\pi}{2})}V^{\top})
\end{equation}


 

## Reconstruction

So now we have determined $E$ we can get $R$ and $T$ (up to a scale factor). 

What next? How do we reconstruct the 3D points?

Firstly we must take the scale factor into account.

$||E||=||T||=\gamma\in \mathbb{R}+$

So a single point in 3D $X^j$ can be determined by 

\begin{equation}
	\lambda_2^jx^j_2=\lambda_1^jRx^j_1 + \gamma T 
\end{equation}

The unknown scale parameters are $\lambda^j_i$. We've seen previously how we can get rid of one of these. Multiply across by 

$x^j_{2\times}$.
\begin{equation}
	\lambda_2^jx^j_{2\times}x^j_2=\lambda_1^jx^j_{2\times}Rx^j_1 + \gamma x^j_{2\times}T 
\end{equation}
    
    


This gives us:
\begin{equation}
	\lambda_1^jx^j_{2\times}Rx^j_1 + \gamma x^j_{2\times}T = 0
\end{equation}

Which we can show as a linear system of the following sort.
\begin{equation}
	\begin{bmatrix}
	x^j_{2\times}Rx^j_1 & x^j_{2\times} T \end{bmatrix} 
	\begin{bmatrix}
	\lambda_1^j \\
	\gamma 
	\end{bmatrix}= 0
\end{equation}

So we can make a whole vector out of $$
	\begin{bmatrix}
	\lambda_1^j \\
	\gamma 
	\end{bmatrix}$$ 
which we will call $\vec{\lambda}$.
	
\begin{equation}
	\vec{\lambda}=
	\begin{bmatrix}
	\lambda_1^1 \\
	\lambda_1^2 \\
	.\\
	.\\
	.\\
	\lambda_1^n \\
	\gamma 
	\end{bmatrix} \in \mathbb{R}^{n+1}
\end{equation}


 

So we can solve the system $M\vec{\lambda}=0$
where 


\begin{equation}
	M \equiv 
	\begin{bmatrix}
	x^1_{2\times}Rx^1_1 & 0 & 0 & 0 & 0 & x^1_{2\times}T\\
    0 & x^2_{2\times}Rx^2_1 &  0 & 0 & 0 & x^2_{2\times}T\\
	0 & 0 & \ddots & 0 & 0& \vdots\\
	0 & 0 & 0 & x^{n-1}_{2\times}Rx^{n-1}_1 &   0 & x^{n-1}_{2\times}T\\
	0 & 0 & 0 & 0 & x^{n}_{2\times}Rx^{n}_1 &    x^{n}_{2\times}T
	\end{bmatrix}
	\end{equation}

$3n \times n+1$
	
Once again, we are looking for a vector in the null space but in truth there is unlikely to be a perfect null space.

Instead we can do an eigen vector decomposition of $M^{\top}M$ and $\vec{\lambda}$ will be the vector associated with the smallest eigen value.

Again this is only defined up to a global scale.

In [627]:
def skew(vec):
    return np.array([[0,-vec[2], vec[1]],
                     [vec[2],0,-vec[0]],
                     [-vec[1],vec[0],0]])
x1 = np.array([-0.3846,0.3846,1])
x2 = np.array([0.1465,-0.0709,1])
lamb1 = 1.3
lamb2 = 3.08826859

#print((rot[0:3,0:3] @ x1))
print(( (lamb1*x1)))
print((R2 @ (lamb2*x2)) + T2)

print(rot[0:3,0:3].T @ ((lamb1*x1) - T[0:3,3]))

print(lamb2*x2)
#print((lamb1*(rot[0:3,0:3] @ x1 - T[3,0:3])/(lamb2*x2))

#print(rot @T)

#print(np.linalg.inv(rot@T))


#rotInv = rotTInv[0:3,0:3]    
#print(np.linalg.det(rotInv))

[-0.5  0.5  1.3]
[-0.5 -1.5 -5.3]
[ 0.452 -0.219  3.088]
[ 0.452 -0.219  3.088]


In [443]:


np.set_printoptions(precision=4, suppress=True)



def skew(vec):
    return np.array([[0,-vec[2], vec[1]],
                     [vec[2],0,-vec[0]],
                     [-vec[1],vec[0],0]])




x2x = np.zeros((8,3,3))
x1 = np.zeros((8,3,1))
x2xT = np.zeros((3,1))

#print(c)
M = np.zeros((24,9))
for i in range (0,8):
    x2x[i] = skew(np.array([c[i,0,0],c[i,0,1],1]))
    x1[i,:,:] = np.array([[c[i,1,0]],[c[i,1,1]],[1]])
    #print(x2x[i])
    M[3*i:(3*(i+1)), i:(i+1)] = x2x[i] @ rot[0:3,0:3] @ x1[i]    
    x2xT[:,0] = x2x[i] @ T[0:3,3]
    #print(x2xT)
    M[3*i:3*(i+1), 8:9 ] = x2xT

#print(M)
MTM = M.transpose() @ M
#print(f'rank of M: {np.linalg.matrix_rank(M)}')

U, S, Vh = np.linalg.svd(M)


lamb = Vh[8,:]
#print(S)
#print(f'T: is {T[0:3,3]}') 
#print(np.linalg.norm(T[0:3,3]))

#print(Vh)

#print(rot[0:3,0:3])b
print(lamb/lamb[8])
print(world_coord[:,2])
print(S)

#print(1.3*M[:,0]/M[:,8])


[3.0883 2.049  2.9151 1.9856 1.8758 2.2187 1.7758 2.0722 1.    ]
[1.3 2.5 1.5 2.4 2.7 1.9 2.7 2.3]
[5.2725 0.9549 0.926  0.8539 0.8341 0.7339 0.7059 0.3753 0.    ]


In [447]:


np.set_printoptions(precision=4, suppress=True)



def skew(vec):
    return np.array([[0,-vec[2], vec[1]],
                     [vec[2],0,-vec[0]],
                     [-vec[1],vec[0],0]])




x2x = np.zeros((8,3,3))
x1 = np.zeros((8,3,1))
x2xT = np.zeros((3,1))

#print(c)
M = np.zeros((24,9))
for i in range (0,8):
    x2x[i] = skew(np.array([c[i,1,0],c[i,1,1],1]))
    x1[i,:,:] = np.array([[c[i,0,0]],[c[i,0,1]],[1]])
    #print(x2x[i])
    M[3*i:(3*(i+1)), i:(i+1)] = x2x[i] @ rot[0:3,0:3].T @ x1[i]    
    x2xT[:,0] = x2x[i] @ T[0:3,3]
    #print(x2xT)
    M[3*i:3*(i+1), 8:9 ] = x2xT

#print(M)
MTM = M.transpose() @ M
#print(f'rank of M: {np.linalg.matrix_rank(M)}')

U, S, Vh = np.linalg.svd(M)


lamb = Vh[8,:]
#print(S)
#print(f'T: is {T[0:3,3]}') 
#print(np.linalg.norm(T[0:3,3]))

#print(Vh)

#print(rot[0:3,0:3])b
print(lamb/lamb[8])
print(world_coord[:,2])
print(S)

#print(1.3*M[:,0]/M[:,8])


[ -1.6796  -0.0856  -1.7491  -1.3314   0.0866 -11.0633  -0.3054  -1.3013
   1.    ]
[1.3 2.5 1.5 2.4 2.7 1.9 2.7 2.3]
[3.3247 0.9587 0.9488 0.8548 0.8393 0.7609 0.7063 0.6049 0.2893]


## Defined up to scale


What this means is that if the camera views are a known distance apart then we can calculate the scale of objects.

But if they are not then we cannot tell if they are, for example 5cm apart and we are looking at minatures or 1m apart and we are looking at objects 20 times as large.



![](images/smallFaraway.jpg)

## Let's take stock
So if we get 8 _good_ correspondences, we can determine the Rotation of the camera. 

We can kind of determine the translation in that 3D points will be related to $T$ by some multiple.

From there we can determine the 3D points themselves.

We can do this with more than the 8 points. 

We can do it with any points we have correspondences for but obviously the reconstruction will only be as good as the quality of the correspondences.

## What should our concerns be?
What if the correspondences are not good? 

- This can lead to degenerate configurations which we will talk about next.
- The points may not be degenerate but have all sorts of noise and errors. There are therefore many algorithms for best minimising this. 


We assumed earlier that we knew the camera intrinsic parameters, what if we don't?  

In most cases in the automotive sector you will know the intinisic parameters as you will have calibrated as a separate step. 

But in the case where you know nothing of the cameras there is an extension to the Essential matrix, called the Fundamental matrix. 

More later. There will even be a song about it.

## Sparse Reconstruction

What can we tell about the world after it?

Emmm, errr, wellll, not a lot really. 

Knowing a few points in the 3D world is not going to paint you a nice picture of the world.

Even with one hundred points, what do we do? 

Join the dots? 

The dots aren't numbered, so it's not child's play.

## Degenerate Configurations

So the eight point algorithm is often stated as providing a unique solution up to a scale factor for eight 3D points in general position.

The _in general position_ can often be missed in amongst the other detail.

In general position, means that the points must not lie on certain 2D surfaces, often called critical surfaces.

When a configuration does not give us a unique solution for a particular class of transformations, we call this degenerate.

Importantly, this refers to the configuration but also the transformations involved.

So one transformation's degenerate configuration may be the ideal for another transformation.

### Points on a plane in 2D

Many of the degenerate configurations for the 8-point algorithm refer to surfaces that are described by quadratic equations. (Sometimes referred to as quadric surfaces).

Most of them don't really show up in real world situations but one that definitely does is the situation that the 8-points lie in a 2D plane in the 3D world.

For example if all of the 8-points are on the wall of a building for both views then the 8-point algorithm will fail.
Line markings for a parking place all lie on a plane.

Shortly we will see that if we know that all points fall on a plane then we can turn this degenerate configuration into an advantage, and get away with a 4-point algorithm.

### No Translation
This is the situation where the two camera views have the exact same camera center, i.e. the camera centers are said to be coincident.

So either the camera hasn't moved or it has undergone a pure rotation.

Either way, recovery of 3D geometry is not possible in this configuration.

The images might be nice for generating a panorama though.

## Planar Homographies

Unlike the previous case where we want eight points in general position, here we want four points and they must lie on a plane in the 3D world.

One of the ways to describe a plane is with a normal vector. 

This is a vector that is normal to the plane, i.e. at $90^o$ to the plane.

Once again we are using the idea of a null space. 

In a 3D world a single vector makes up just one dimension. 


## Normal Vector

The normal vector only meets the plane at the origin in the 3D coordinates. 

Therefore any 3D vector that is in the null space of the normal vector space is in the plane.

You can think of this as describing the plane as everything that the normal vector is not. i.e. vectors that have absolutely nothing in common with the normal vector are in the plane. 

By _in common_ here we mean that if projected onto the normal vector it would result in some non-zero amount.

But vectors in the plane, when projected onto the normal vector result in zero.


## Plane not passing through origin

So previously we assumed that the plane passes through the origin and then the normal vector meets the plane only at the origin.

But this is a little restrictive as we want to discuss planes that do not pass through the origin.

This is relatively straight forward, we simply move the plane along the normal vector some distance $d$. 

Now if we project any vector on the plane onto the normal vector, it will result in the scalar value $d$ rather than zero.

So we can place the origin at one of the camera centers, which is useful.



## Equation of a plane

\begin{equation}
	N^{\top}\mathbf{X}_1=d
\end{equation}

or

\begin{equation}
	\frac{1}{d}N^{\top}\mathbf{X}_1=1
\end{equation}


This is w.r.t camera one.
In terms of camera two's center this is 
\begin{equation}
	\mathbf{X}_2 = R\mathbf{X}_1 + T
\end{equation}

There is a clever trick we can use here. $T$ is the same as $1\times T$ and we see from the second equation for a plane above  that we have something useful that is equal to 1, and we can substitute it in.

\begin{equation}
	\mathbf{X}_2 = R\mathbf{X}_1 + T\frac{1}{d}N^{\top}\mathbf{X}_1
\end{equation}

## Homography Matrix

You need to be careful to consider the shapes of the vectors to know which side you can multiply to get the right behaviour.
We now see that $\mathbf{X}_1$ is common so we can take it out as a common factor.

\begin{equation}
	\mathbf{X}_2 = \left(R + \frac{1}{d}TN^{\top}\right)\mathbf{X}_1 \equiv H\mathbf{X}_1
\end{equation}

$H$ is a $3\times3$ matrix called a homography matrix,
and it relates the 3D points that lie on a 2D plane to eachother in terms of the two camera centers, i.e. The same point in two different coordinate frames.

\begin{equation}
	\mathbf{X}_2 = H\mathbf{X}_1
\end{equation}


## Homography Matrix for 2D
Of course we don't usually know the 3D coordinates. We've been here before, we know the 2D so we can say\\


\begin{equation}
	\lambda_2\mathbf{x}_2 = \lambda_1H\mathbf{x}_1
\end{equation}

Again as the $\lambda s$ show, the 2D coordinates are related up to scale by the homography matrix.

The homography matrix encodes the camera extrinsic parameters and parameters describing the plane.

Just like with the points in general position, we can get rid of $\lambda s$ by multiplying across by $\mathbf{x}_{2\times}$
\begin{equation}
	\mathbf{x}_{2\times}H\mathbf{x}_{1} = 0
\end{equation}

This is called the planar epipolar constraint or planar homography constraint. 
Note the subtle difference, $\mathbf{x}_{2\times}$ instead of $\mathbf{x}_{2}$

We can use our stacking operators as before, to make a vector $h_s$

$$h_s= \begin{bmatrix}
h_{11}\\
h_{21}\\
h_{31}\\
h_{12}\\
h_{22}\\
h_{32}\\
h_{13}\\
h_{23}\\
h_{33}
 \end{bmatrix} $$



But $\mathbf{x}_{2\times}$ is a $3\times3$ skew symmetric matrix.

So $\mathbf{a}$ this time is a $9\times3$ matrix instead of a vector.

\begin{equation}
	\mathbf{a}\equiv \mathbf{x}_1 \otimes \mathbf{x_{2\times}}
\end{equation}

\begin{equation}
	\mathbf{a}^{\top}h_s = 0
\end{equation}

## The four point algorithm
If we have $n$ pairs of 2D point correspondances where $n\geq4$ then each point pair $\mathbf{x}^j_1,\mathbf{x}^j_2 $ can be represented by matrix $\mathbf{a}^j$.

These can be put together in a combined matrix $\mathbf{A}$ as follows.
\begin{equation}
\mathbf{A}\equiv (\mathbf{a}^1,...,\mathbf{a}^n)^{\top}
\end{equation}

Which is a $3n\times9$ matrix.

And this gives us a full system to solve.

\begin{equation}
	\mathbf{A}h_s=0
\end{equation}

Just like with the essential matrix, the homography matrix can be estimated up to a scale factor.
So the consise algorithm is

1. Compute the matrix $\mathbf{A}$ for the four points
2. Compute a solution to $h_s$ for equation above using SVD and taking the column vector with the smallest singular value.
3. Extract the motion parameters from the homography matrix $H=R+\frac{1}{d}TN^{\top}$.


$H$ can be decomposed into $R, N$ and $\frac{T}{d}$.
[Inra, Ezio Malis, Manuel Vargas Page 8](https://hal.inria.fr/inria-00174036v3/document)

Once again we see that we can reconstruct the translation up to a scale. 
In this case it is scaled by the distance to the plane.
We can compute the 3D coordinates as we did for the 8-point algorithm.

##  Effect of the camera matrix $K$
The essential matrix worked on the principle that we know the camera intrinsic paramters and further that we assume no skew and we give everything in units of focal length leading to the canonical camera matrix $K =I$ where $I$ is the $3\times3$ identity matrix.

It's reasonable to think of this as not so much knowing the camera matrix as pretending it doesn't exist.

However the camera matrix, if it is not the identity, will have an effect on the projection of 3D coordinates.



## If we know $K$
If we have separately calculated the calibration matrix for the camera and therefore know $K$, all we need do is transform our pixel coordinates to normalised pixel coordinates using $K^{-1}$.

\begin{equation}
	\mathbf{x}=K^{-1}\mathbf{x}'
\end{equation}

We can do this with  all point correspondences and then we are at our starting point for either our 4 or 8-point algorithm.

This is the most likely scenario in the automotive setting. 


## The uncalibrated camera
Imagine the situation where we have two views but no information about the camera.

This might be the case with archive film.

It is also the case where we take two images of a location from the internet from two different photographers.

i.e. two views, two cameras, no information.

This is attempting reconstruction from un-calibrated views.



## The Fundamental Matrix

Lets assume the same camera has taken both views, so just one camera matrix $K$.

We can rework the epipolar constraint as follows.

\begin{equation}
	x_2^{\top}T_{\times}Rx_1=0 \to x_2^{'}K^{-\top}T_{\times}RK^{-1}x'_1=0
\end{equation}

This gives rise to the fundamental matrix $F$

\begin{equation}
	F = K^{-\top}T_{\times}RK^{-1} \text{\quad  or \quad } F = K^{-\top}EK^{-1}
\end{equation}

\begin{equation}
	 x_2^{'T}K^{-\top}T_{\times}RK^{-1}x'_1=0
\end{equation}



## SVD of the fundamental matrix $F$
$K$ is invertible so $F$ will have the same rank as $E$.

So we can use the exact same method of SVD of $F$ as for $E$, i.e.
\begin{equation}
	\text{SVD } F = U\Sigma V^{\top}. 
\end{equation}
With 
\begin{equation}
	\Sigma = \begin{bmatrix}
	\sigma_1 & 0 & 0\\
0 & \sigma_2 & 0\\
0 & 0 &0
 \end{bmatrix}
\end{equation}

Many texts and courses start with the Fundamental matrix and break it down to the Camera matrix and Essential matrix.

However chronologically the Essential matrix came first.

Either way the method for solving them is the same.

Now, about that song.
[Fundamental Matrix Song](https://www.youtube.com/watch?v=DgGV3l82NTk)

## Some considerations on $F$
Note that $\sigma_1$ and $\sigma_2$ are not necessarily the same size as in the Essential Matrix.

Getting $F$ may be similar to getting $E$ but from there things get harder. The decomposition is not so easy, i.e. we cannot easily separate the intrinsic and extrinsic parameters.

We can determine reconstructions up to a projective reconstruction.

The take away is, **try to find the camera intrinsics separately.**